Importing all the libaries

In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

Scrape a CSV file from the spreadsheet

In [193]:
# get csv from to df https://docs.google.com/spreadsheets/d/1ZafspjnRJuDjLRKotQ8awLTGcf3RLxrBEh2JtqRGh0Y/edit?usp=sharing
df = pd.read_csv('https://docs.google.com/spreadsheets/d/1ZafspjnRJuDjLRKotQ8awLTGcf3RLxrBEh2JtqRGh0Y/export?format=csv')
df.head()

,Status,School,Program,Average,Date Accepted,Type (101/105),Discord,Other
0,Accepted,Brock University,Sport Management,89% (final grade 12 marks),"December 8, 2021",101,Denim#1627,NaN
1,Accepted,Brock University,Public Health (co-op),84%,December 17th,105F,Samantha#8888,French Baccalaureate
2,Accepted,Brock University,Bachelor of Accounting + Co-op,95.1,January 10th,101,Billion Dollar#9365,NaN
3,Accepted,Brock University,BA Honours: Interactive Arts & Science,"Gr.11 94%, first sem gr12 92%",Dec 8 2021,101,percn#0001,NaN
4,Accepted,Carelton University,Computer Systems Engineering,95%,NaN,NaN,Sarina#7691,NaN


Clean Data

In [194]:
# Let's remove all the columns that we don't need
df = df.drop(columns=['Discord', 'Other', 'Date Accepted']).reset_index(drop=True)
# Remove all the '%" in the average column
df['Average'] = df['Average'].str.replace('%', '')
# remove all + in the average column
df['Average'] = df['Average'].str.replace('+', '')
# Delete rows where average  NaN
df = df.dropna(how = 'any', subset=['Average'])

df.head(50)

C:\Users\MOHAMM~1\AppData\Local\Temp/ipykernel_3616/3305083995.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Average'] = df['Average'].str.replace('+', '')


,Status,School,Program,Average,Type (101/105)
0,Accepted,Brock University,Sport Management,89 (final grade 12 marks),101
1,Accepted,Brock University,Public Health (co-op),84,105F
2,Accepted,Brock University,Bachelor of Accounting + Co-op,95.1,101
3,Accepted,Brock University,BA Honours: Interactive Arts & Science,"Gr.11 94, first sem gr12 92",101
4,Accepted,Carelton University,Computer Systems Engineering,95,NaN
5,Accepted,Carelton University,Bachelor of Computer Science,98.3,101
6,Accepted,Carelton University,Bachelor of Computer Science (co-op),91,101
7,Accepted,Carleton,Computer science (software engineering),94,101
8,Accepted,Carleton,BCS,mid 90's,105F
9,Accepted,Carleton,Coms and Media,"95 for Top 5, 92 for Top 6",101


UTSG CS

In [195]:
#Get all utsg

# get df where utsg is there
df_utsg1 = df[df['School'].str.contains('UTSG', na=False, case=False)]
# get df where the school has toronto in the name
df_utsg2 = df[df['School'].str.contains('Toronto', na=False, case=False)]
# get uoft
df_utsg3 = df[df['School'].str.contains('UOFT', na=False, case=False)]
#get u of t
df_utsg4 = df[df['School'].str.contains('U of T', na=False, case=False)]
# combine all 3
df_utsg = pd.concat([df_utsg1, df_utsg2, df_utsg3, df_utsg4])

# Get all cs within utsg not case sensitive
df_utsgcs1 = df_utsg[df_utsg['Program'].str.contains('Computer Science', na=False, case=False)]
df_utsgcs2 = df_utsg[df_utsg['Program'].str.contains('CS', na=False, case=False)]
df_utsgcs3 = df_utsg[df_utsg['Program'].str.contains('comp sci', na=False, case=False)]

df_utsg_cs = pd.concat([df_utsgcs1, df_utsgcs2, df_utsgcs3])

# drop all rows where school has scarbgrough in the name
df_utsg_cs = df_utsg_cs[~df_utsg_cs['School'].str.contains('Scarborough', na=False, case=False)]
# drop all rows where program has math in the name
df_utsg_cs = df_utsg_cs[~df_utsg_cs['Program'].str.contains('Math', na=False, case=False)]
# get all accepted
df_utsg_cs_accepted = df_utsg_cs[df_utsg_cs['Status'] == 'Accepted'].reset_index(drop=True)
df_utsg_cs_accepted101 = df_utsg_cs_accepted[df_utsg_cs_accepted['Type (101/105)'] == '101'].reset_index(drop=True)
df_utsg_cs_accepted105 = df_utsg_cs_accepted[df_utsg_cs_accepted['Type (101/105)'].str.contains('105')].reset_index(drop=True)

#get average of df_utsg_cs_accepted average column
df_utsg_cs_accepted['Average'] = df_utsg_cs_accepted['Average'].astype(float)
df_utsg_cs_accepted105['Average'] = df_utsg_cs_accepted105['Average'].astype(float)
df_utsg_cs_accepted101['Average'] = df_utsg_cs_accepted101['Average'].astype(float)

print("Admission Average: " + str(df_utsg_cs_accepted['Average'].mean()))
print("Admission Average 101: " + str(df_utsg_cs_accepted101['Average'].mean()))
print("Admission Average 105: " + str(df_utsg_cs_accepted105['Average'].mean()))


Admission Average: 97.93986956521738
Admission Average 101: 97.87984210526315
Admission Average 105: 98.225


Waterloo Software Engineering

In [196]:
df_uw1 = df[df['School'].str.contains('UW', na=False, case=False)]
df_uw2 = df[df['School'].str.contains('University of Waterloo', na=False, case=False)]
df_uw3 = df[df['School'].str.contains('Waterloo', na=False, case=False)]

df_uw = pd.concat([df_uw1, df_uw2, df_uw3])

df_uw_se1 = df_uw[df_uw['Program'].str.contains('Software Engineering', na=False, case=False)]
df_uw_se2 = df_uw[df_uw['Program'].str.contains('SE', na=False, case=False)]
df_uw_se3 = df_uw[df_uw['Program'].str.contains('software eng', na=False, case=False)]
df_uw_se4 = df_uw[df_uw['Program'].str.contains('soft eng', na=False, case=False)]

df_uw_se = pd.concat([df_uw_se1, df_uw_se2, df_uw_se3]).reset_index(drop=True)

# get all the accepted
df_uw_se_accepted = df_uw_se[df_uw_se['Status'] == 'Accepted'].reset_index(drop=True)
df_uw_se_accepted101 = df_uw_se_accepted[df_uw_se_accepted['Type (101/105)'] == '101'].reset_index(drop=True)
df_uw_se_accepted105 = df_uw_se_accepted[df_uw_se_accepted['Type (101/105)'].str.contains('105')].reset_index(drop=True)

# get the averages
df_uw_se_accepted['Average'] = df_uw_se_accepted['Average'].astype(float)
df_uw_se_accepted105['Average'] = df_uw_se_accepted105['Average'].astype(float)
df_uw_se_accepted101['Average'] = df_uw_se_accepted101['Average'].astype(float)

# print the averages
print("Admission Average: " + str(df_uw_se_accepted['Average'].mean()))
print("Admission Average 101: " + str(df_uw_se_accepted101['Average'].mean()))
print("Admission Average 105: " + str(df_uw_se_accepted105['Average'].mean()))
df_uw_se_accepted.head(50)



Admission Average: 98.5090909090909
Admission Average 101: 99.2
Admission Average 105: 91.6


,Status,School,Program,Average,Type (101/105)
0,Accepted,University of Waterloo,Software Engineering,99.2,101
1,Accepted,Waterloo,Software Engineering,99.0,101
2,Accepted,Waterloo,Software Engineering,99.7,101
3,Accepted,University of Waterloo,Software Engineering,99.2,101
4,Accepted,Waterloo,SE,97.8,101
5,Accepted,University of Waterloo,Software Engineering,99.2,101
6,Accepted,Waterloo,Software Engineering,99.0,101
7,Accepted,Waterloo,Software Engineering,99.7,101
8,Accepted,Waterloo,Software engineer,91.6,105D
9,Accepted,waterloo,software eng,100.0,101


Waterloo Computer Engineering

In [197]:
df_uw_ce1 = df_uw[df_uw['Program'].str.contains('Computer Engineering', na=False, case=False)]
df_uw_ce2 = df_uw[df_uw['Program'] == 'CE']
df_uw_ce5 = df_uw[df_uw['Program'] == 'ce']
df_uw_ce3 = df_uw[df_uw['Program'].str.contains('computer eng', na=False, case=False)]
df_uw_ce4 = df_uw[df_uw['Program'].str.contains('comp eng', na=False, case=False)]

df_uw_ce = pd.concat([df_uw_ce1, df_uw_ce2, df_uw_ce3, df_uw_ce4, df_uw_ce5]).reset_index(drop=True)

#get all accepted
df_uw_ce_accepted = df_uw_ce[df_uw_ce['Status'] == 'Accepted'].reset_index(drop=True)
df_uw_ce_accepted101 = df_uw_ce_accepted[df_uw_ce_accepted['Type (101/105)'] == '101'].reset_index(drop=True)
df_uw_ce_accepted105 = df_uw_ce_accepted[df_uw_ce_accepted['Type (101/105)'].str.contains('105')].reset_index(drop=True)

df_uw_ce_accepted.head(50)
#get the averages
df_uw_ce_accepted['Average'] = df_uw_ce_accepted['Average'].astype(float)
df_uw_ce_accepted105['Average'] = df_uw_ce_accepted105['Average'].astype(float)
df_uw_ce_accepted101['Average'] = df_uw_ce_accepted101['Average'].astype(float)

# print the averages
print("Admission Average: " + str(df_uw_ce_accepted['Average'].mean()))
print("Admission Average 101: " + str(df_uw_ce_accepted101['Average'].mean()))
print("Admission Average 105: " + str(df_uw_ce_accepted105['Average'].mean()))

df_uw_ce_accepted.head(50)



Admission Average: 95.40724347826088
Admission Average 101: 95.40724347826088
Admission Average 105: nan


,Status,School,Program,Average,Type (101/105)
0,Accepted,University of Waterloo,Computer Engineering,94.0000,101
1,Accepted,University of Waterloo,Computer Engineering,94.5000,101
2,Accepted,Waterloo,Computer Engineering,98.2000,101
3,Accepted,Waterloo,Computer Engineering,97.5000,101
4,Accepted,waterloo,computer engineering,95.0000,101
5,Accepted,University of Waterloo,Computer Engineering,94.0000,101
6,Accepted,waterloo,computer engineering,92.8333,101
7,Accepted,waterloo,computer engineering,98.7500,101
8,Accepted,Waterloo,Computer Engineering,97.5000,101
9,Accepted,Waterloo,Computer Engineering,93.0000,101


UW Tron

In [198]:
df_uw_tron1 = df_uw[df_uw['Program'].str.contains('Tron', na=False, case=False)]
df_uw_tron2 = df_uw[df_uw['Program'].str.contains('Mechatronics', na=False, case=False)]

# combine
df_uw_tron = pd.concat([df_uw_tron1, df_uw_tron2]).reset_index(drop=True)

# get accepted
df_uw_tron_accepted = df_uw_tron[df_uw_tron['Status'] == 'Accepted'].reset_index(drop=True)
df_uw_tron_accepted101 = df_uw_tron_accepted[df_uw_tron_accepted['Type (101/105)'] == '101'].reset_index(drop=True)
df_uw_tron_accepted105 = df_uw_tron_accepted[df_uw_tron_accepted['Type (101/105)'].str.contains('105')].reset_index(drop=True)

# get averages
df_uw_tron_accepted['Average'] = df_uw_tron_accepted['Average'].astype(float)
df_uw_tron_accepted105['Average'] = df_uw_tron_accepted105['Average'].astype(float)
df_uw_tron_accepted101['Average'] = df_uw_tron_accepted101['Average'].astype(float)

# print averages
print("Admission Average: " + str(df_uw_tron_accepted['Average'].mean()))
print("Admission Average 101: " + str(df_uw_tron_accepted101['Average'].mean()))
print("Admission Average 105: " + str(df_uw_tron_accepted105['Average'].mean()))

df_uw_tron.head(50)


Admission Average: 97.26727272727273
Admission Average 101: 97.26727272727273
Admission Average 105: nan


,Status,School,Program,Average,Type (101/105)
0,Accepted,University of Waterloo,Mechatronics Engineering,99.5,101
1,Accepted,University of Waterloo,Mechatronics Engineering,95.5,101
2,Accepted,University of Waterloo,Mechatronics Engineering,95.5,101
3,Accepted,University of Waterloo,Mechatronics Engineering (Hons. Co-op),98.17,101
4,Accepted,University of Waterloo,Mechatronics Engineering,98.3,101
5,Accepted,University of Waterloo,Mechatronics Engineering,99.5,101
6,Accepted,University Waterloo,Mechatronics engineering,96,101
7,Accepted,University of Waterloo,Mechatronics Engineering,95.5,101
8,Accepted,University of Waterloo,Mechatronics Engineering,95.5,101
9,Accepted,University of Waterloo,Mechatronics Engineering (Hons. Co-op),98.17,101


UW MGMNT

In [199]:
df_uw_mgmnt = df_uw[df_uw['Program'].str.contains('Management', na=False, case=False)]

df_uw_mgmnt_accepted = df_uw_mgmnt[df_uw_mgmnt['Status'] == 'Accepted'].reset_index(drop=True)
df_uw_mgmnt_accepted101 = df_uw_mgmnt_accepted[df_uw_mgmnt_accepted['Type (101/105)'] == '101'].reset_index(drop=True)
df_uw_mgmnt_accepted105 = df_uw_mgmnt_accepted[df_uw_mgmnt_accepted['Type (101/105)'].str.contains('105')].reset_index(drop=True)

#get the averages
df_uw_mgmnt_accepted['Average'] = df_uw_mgmnt_accepted['Average'].astype(float)
df_uw_mgmnt_accepted105['Average'] = df_uw_mgmnt_accepted105['Average'].astype(float)
df_uw_mgmnt_accepted101['Average'] = df_uw_mgmnt_accepted101['Average'].astype(float)

#print the averages
print("Admission Average: " + str(df_uw_mgmnt_accepted['Average'].mean()))
print("Admission Average 101: " + str(df_uw_mgmnt_accepted101['Average'].mean()))
print("Admission Average 105: " + str(df_uw_mgmnt_accepted105['Average'].mean()))

df_uw_mgmnt_accepted.head(50)

Admission Average: 96.24
Admission Average 101: 96.24
Admission Average 105: nan


,Status,School,Program,Average,Type (101/105)
0,Accepted,University of Waterloo,Management Engineering,96.0,101
1,Accepted,University of Waterloo,Management Engineering,96.0,101
2,Accepted,Waterloo,Management Eng,98.2,101
3,Accepted,waterloo,management engineering,95.2,101
4,Accepted,Waterloo,Management Eng,95.8,101


UOFT Eng Sci

In [200]:
df_utsg_engsci1 = df_utsg[df_utsg['Program'].str.contains('Engineering Science', na=False, case=False)]
df_utsg_engsci2 = df_utsg[df_utsg['Program'].str.contains('EngSci', na=False, case=False)]
df_utsg_engsci3 = df_utsg[df_utsg['Program'].str.contains('Eng sci', na=False, case=False)]

#combine
df_utsg_engsci = pd.concat([df_utsg_engsci1, df_utsg_engsci2, df_utsg_engsci3]).reset_index(drop=True)

# get accepted
df_utsg_engsci_accepted = df_utsg_engsci[df_utsg_engsci['Status'] == 'Accepted'].reset_index(drop=True)
df_utsg_engsci_accepted101 = df_utsg_engsci_accepted[df_utsg_engsci_accepted['Type (101/105)'] == '101'].reset_index(drop=True)
df_utsg_engsci_accepted105 = df_utsg_engsci_accepted[df_utsg_engsci_accepted['Type (101/105)'].str.contains('105')].reset_index(drop=True)

# get average of 101
df_utsg_engsci_accepted['Average'] = df_utsg_engsci_accepted['Average'].astype(float)
df_utsg_engsci_accepted101['Average'] = df_utsg_engsci_accepted101['Average'].astype(float)
df_utsg_engsci_accepted105['Average'] = df_utsg_engsci_accepted105['Average'].astype(float)

# print averages
print("Admission Average: " + str(df_utsg_engsci_accepted['Average'].mean()))
print("Admission Average 101: " + str(df_utsg_engsci_accepted101['Average'].mean()))
print("Admission Average 105: " + str(df_utsg_engsci_accepted105['Average'].mean()))




df_utsg_engsci.head(50)

Admission Average: 97.88749999999999
Admission Average 101: 98.25
Admission Average 105: 96.8


,Status,School,Program,Average,Type (101/105)
0,Accepted,University of Toronto,Engineering Science,98,105F
1,Accepted,University of Toronto,Engineering Science,95.6,105F
2,Accepted,uoft,engineering science,98.2,101
3,Accepted,UofT,Engineering Science,98,101
4,Accepted,U of T,Engineering Science,98.3,101
5,Accepted,uoft,engsci,100.0,101
6,Accepted,UofT,Eng Sci,99,101
7,Accepted,U of T,Eng Sci,96,101
